In [1]:
// nuget references
#r "nuget: FSharp.Stats, 0.5.1-preview.1"
#r "nuget: Plotly.NET.Interactive, 4.2.1"
#r "nuget: FSharp.Data, 6.3.0"
#r "nuget: Cytoscape.NET.Interactive, 0.2.0"

open FSharp.Stats
open Plotly.NET
open Plotly.NET.StyleParam
open Plotly.NET.LayoutObjects
open FSharp.Data
open Cytoscape.NET
open System


//FSharp.Stats.ServiceLocator.setEnvironmentPathVariable (@"C:\Users\bvenn\source\repos\FSharp.Stats\lib")
//FSharp.Stats.Algebra.LinearAlgebra.Service()

//axis styling extension module
module Chart = 
    let myAxis name = LinearAxis.init(Title=Title.init name,Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=false,ShowLine=true)
    let withAxisTitles x y chart = 
        chart 
        |> Chart.withTemplate ChartTemplates.lightMirrored
        |> Chart.withXAxis (myAxis x) 
        |> Chart.withYAxis (myAxis y)

type Category =
    | Beer
    | Beverage
    | Coffee
    | Other
    with 
        static member FromString (s: string) =
            match s with
            | "Beer" -> Beer
            | "Beverage" -> Beverage
            | "Coffee" -> Coffee
            | _ -> Other

type Order = {
    DateTime    : System.DateTime
    Name        : string
    Gender      : char
    Product     : string
    Price       : float
    Department  : string
    Category    : Category
    Amount      : int
    } with
        static member Create time (name: string) gender product price department category amount = {
            DateTime  = time
            Name      = name
            Gender    = gender
            Product   = product
            Price     = price
            Department= department
            Category  = category
            Amount    = amount
            }

let data = 
    let read =
        CsvFile
            //.Load(@"..\data\coffeedata.txt")
            .Load(@"C:\Users\bvenn\source\repos\brewing-discoveries-workshop\data\coffeedata.txt")
            .Cache()
    read.Rows
    |> Seq.map (fun row -> 
        Order.Create
            (System.DateTime.ParseExact((row.GetColumn "DateTime"),"dd/MM/yyyy HH:mm:ss",null))
            (row.GetColumn "Name")
            (row.GetColumn "Gender" |> char)
            (row.GetColumn "Product")
            (row.GetColumn "Price" |> float) 
            (row.GetColumn "Department")
            ((row.GetColumn "Category") |> Category.FromString)
            (row.GetColumn "Amount" |> int)
        )
    |> Array.ofSeq

let getDepartmentColor (department: string) = 
    match department with 
    | "Breakroom Bandits" -> "#2b3ae9"
    | "Genesis" -> "#f7da41"
    | "We Tried" -> "#008b66"
    | "No Lucks Given" -> "#987200"
    | "Toon Squad" -> "#ff7f0e"
    | "Rumor Spreaders" -> "#20b2aa"
    | "Risky Biscuits" -> "#a230ed"
    | "Recruitables" -> "#d21102"
    | "Employees of the Moment" -> "#19d3f3"
    | "Chargers" -> "#dea57b"
    | "Kickstarters" -> "#dea57b"
    | _ -> "#8b8b8b"

Installed Packages Cytoscape.NET.Interactive, 0.2.0 FSharp.Data, 6.3.0 FSharp.Stats, 0.5.1-preview.1 Plotly.NET.Interactive, 4.2.1

Loading extensions from `C:\Users\bvenn\.nuget\packages\plotly.net.interactive\4.2.1\interactive-extensions\dotnet\Plotly.NET.Interactive.dll`

Loading extensions from `C:\Users\bvenn\.nuget\packages\cytoscape.net.interactive\0.2.0\interactive-extensions\dotnet\Cytoscape.NET.Interactive.dll`

In [5]:
let persons = 
    data 
    |> Array.map (fun x -> x.Name,getDepartmentColor x.Department) 
    |> Array.distinct
    |> Map.ofArray

let plotSingledayBehavior name =     
    let data =
        data
        |> Seq.filter (fun x -> x.Product <> "Deposit" && x.Product <> "Debit")
        |> Seq.filter (fun x -> if name = "" then true else x.Name = name)
        |> Seq.groupBy (fun x-> x.DateTime.Date)
        |> fun x -> 
            let emptyOrders = seq [for i = 0 to (System.DateTime.Now - System.DateTime(2018,08,30)).Days do yield (System.DateTime(2018,08,30).AddDays(float i), (seq [] :seq<Order>))]
            [|emptyOrders;x|]
        |> Seq.concat 
        |> Seq.sortBy fst
        |> Array.ofSeq              
        |> Seq.skip 7
    let calc (data: seq<DateTime * seq<Order>>)=  
        let post (dat: seq<string*float>) = 
            dat
            |> Array.ofSeq
            |> fun g -> g.[6..] 
            |> Chart.StackedColumn
        
        data
        |> Seq.map (fun (date,day) -> date.ToShortDateString(), day |> Seq.fold (fun acc trade -> acc + (float trade.Amount) ) 0. ) |> post    // (float trade.Amount * trade.Price)   
    let coffee =
        data
        |> Seq.map (fun (date,day) -> date,day |> Seq.filter (fun trade -> trade.Price > 0. && trade.Category = Category.Coffee))
        |> calc
        |> Chart.withTraceInfo "coffee"
    let beer =
        data
        |> Seq.map (fun (date,day) -> date,day |> Seq.filter (fun trade -> trade.Price > 0. && trade.Category = Category.Beer))
        |> calc
        |> Chart.withTraceInfo "beer"
    let beverages =
        data
        |> Seq.map (fun (date,day) -> date,day |> Seq.filter (fun trade -> trade.Price > 0. && trade.Category = Category.Beverage ))
        |> calc
        |> Chart.withTraceInfo "beverages"
    let other =                                                                                                                                                                                              
        data
        |> Seq.map (fun (date,day) -> date,day |> Seq.filter (fun x -> x.Price > 0. && x.Category = Category.Other))
        |> calc
        |> Chart.withTraceInfo "Milk/Food/other"
            
    [coffee;beer;beverages;other]
    |> Chart.combine
    |> Chart.withAxisTitles "" "amount by day and category"
    |> Chart.withSize(1100.,700.)

plotSingledayBehavior "Timo M."

<!-- Plotly chart will be drawn inside this DIV -->

In [7]:

let plotWeekDistByNameAndDay (name: string option) (fromDate: System.DateTime option) (toDate: System.DateTime option) =

    let myAxisRange title (min,max) showtick = LinearAxis.init(Title=Title.init title,Range=StyleParam.Range.MinMax (min,max),Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=true,ShowLine=true,ShowTickLabels=showtick)

    let nameSeq =
        if name.IsNone then 
            if fromDate.IsSome then data |> Array.filter (fun x -> x.DateTime > fromDate.Value && x.DateTime < toDate.Value) else data
            |> Seq.groupBy (fun x -> x.DateTime.DayOfWeek)
            |> Seq.sortBy (fun (key,items) -> key)
            |> fun x -> Seq.append (Seq.tail x) (seq [Seq.head x])
            //|> Seq.map snd
        else 
            if fromDate.IsSome then data |> Array.filter (fun x -> x.DateTime > fromDate.Value && x.DateTime < toDate.Value) else data
            |> Seq.filter (fun x -> x.Name = name.Value)
            |> Seq.groupBy (fun x -> x.DateTime.DayOfWeek)
            |> Seq.sortBy (fun (key,items) -> key)
            |> fun x -> Seq.append (Seq.tail x) (seq [Seq.head x])
            //|> Seq.map snd

    let coffee day (seq: seq<Order>) showtick =
        seq
        |> Seq.filter (fun x -> x.Category = Category.Coffee)
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                                                         
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4b77ad") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "coffee" else "") (-4000000.,83000000.) showtick) |> Chart.withYAxis (Chart.myAxis "#count") |> Chart.withTraceInfo (sprintf "%s coffee" day))  
    let beverage day (seq: seq<Order>) showtick=                                                                                                                                                                                          
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category = Category.Beverage)                                                                                                                                                                     
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#ad504b") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "beverage" else "") (-4000000.,83000000.) showtick)|> Chart.withYAxis (Chart.myAxis "") |> Chart.withTraceInfo (sprintf "%s beverage" day))
    let beer day (seq: seq<Order>) showtick=                                                                                                                                                                                              
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category = Category.Beer)                                                         
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4bad81") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "beer" else "") (-4000000.,83000000.) showtick)|> Chart.withYAxis (Chart.myAxis "")  |> Chart.withTraceInfo (sprintf "%s beer" day))   
    let other day (seq: seq<Order>) showtick=                                                                                                                                                                                              
        seq                                                                                                                                                                                                                 
        |> Seq.filter (fun x-> x.Category = Category.Other) //"Beverage" && x.Category <> "Coffee" && x.Category <> "Deposit" && x.Category <> "Debit" && x.Category <> "TestStuff")
        |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                
        |> (fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#999999") |> Chart.withTemplate ChartTemplates.lightMirrored |> Chart.withXAxis (myAxisRange (if showtick then "Milk/Food/other" else "") (-4000000.,83000000.) showtick)|> Chart.withYAxis (Chart.myAxis "")  |> Chart.withTraceInfo (sprintf "%s other" day))   
            
    nameSeq
    |> Seq.mapi (fun i (day,x) -> 
        let showtick = i=6 
        coffee (day.ToString()) x showtick ,
        beverage (day.ToString()) x showtick ,
        beer (day.ToString()) x showtick )
    |> Array.ofSeq
    |> Array.unzip3
    |> fun (cof,bev,bee) ->
        //let showPlot category cat =
        //    category
            //|> Chart.Stack 1
            //|> Chart.withSize(900.,900.) 
            //|> Chart.withX_Axis (axis (sprintf "DailyDistribution %s - %s" cat name))
        [|
            cof 
            bev 
            bee 
        |]
        |> JaggedArray.transpose
        |> Array.concat
        |> Chart.Grid(7,3)
        |> Chart.withSize (1200.,800.)
        |> Chart.withTitle (sprintf "daily order distribution for %s" (if name.IsNone then "all" else name.Value))
        |> Chart.withMarginSize(Left=50.,Bottom=50.,Top=50.)
        |> fun l -> 
            l 
            |> Chart.withDescription [Giraffe.ViewEngine.HtmlElements.rawText "xAxis=Time, yAxis=count; blue=coffee, red=beverages, green=beer; top=Monday, bottom=Sunday"]
            |> Chart.show 

plotWeekDistByNameAndDay (Some "Benedikt V.") (None) (None)
plotWeekDistByNameAndDay (Some "Benedikt V.") (Some (System.DateTime(2020,04,01))) (Some (System.DateTime(2020,07,01)))
plotWeekDistByNameAndDay (Some "Benedikt V.") (Some (System.DateTime(2023,02,19))) (Some (System.DateTime.Now))


plotWeekDistByNameAndDay (None) None None
plotWeekDistByNameAndDay (None) (Some (System.DateTime(2018,04,01))) (Some (System.DateTime(2020,04,01)))
plotWeekDistByNameAndDay (None) (Some (System.DateTime(2020,04,01))) (Some (System.DateTime(2020,07,01)))


In [ ]:


//how long are the people in the lab?
//how long are the people employed?
//wie viele fehler macht man?
let dailyDist =  
    let myAxisRange title (min,max) = LinearAxis.init(Title=Title.init title,Range=StyleParam.Range.MinMax (min,max),Mirror=StyleParam.Mirror.All,Ticks=StyleParam.TickOptions.Inside,ShowGrid=true,ShowLine=true,ShowTickLabels=true)
    let before5am = data |> Array.filter (fun x -> x.DateTime.TimeOfDay < System.DateTime(2020,1,1,5,0,0).TimeOfDay)
    data
    |> Seq.map (fun x -> DateTime(1970,1,1,x.DateTime.Hour,x.DateTime.Minute,0))                                                                                                                                                                                         
    |> fun x -> Chart.Histogram(x,NBinsX=24,MarkerColor=Color.fromHex "#4b77ad") 
    |> Chart.withXAxis (myAxisRange ("") (-4000000.,83000000.)) 
    |> Chart.withYAxis (Chart.myAxis "#count") 
    |> Chart.withDescription [Giraffe.ViewEngine.HtmlElements.rawText (sprintf "Ticks total: %i<br>Ticks before 5am: %i (%.2f %%)" data.Length before5am.Length (100. * float before5am.Length/float data.Length))]
    |> Chart.withTemplate ChartTemplates.lightMirrored
    |> Chart.withTitle "daily ticking distribution"
    |> Chart.show



data 
|> Array.filter (fun x -> x.DateTime.TimeOfDay > (DateTime(2023,1,1,5,0,0)).TimeOfDay)
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let tmp = 
        orders
        |> Array.groupBy (fun o -> o.DateTime.Date)
        |> Array.choose (fun (date,items) -> 
            let min = items |> Array.map(fun x -> x.DateTime) |> Array.min
            let max = items |> Array.map(fun x -> x.DateTime) |> Array.max 
            if items.Length < 3 || (max - min).Hours = 0 then 
                None 
            else
            Some (float (max - min).Hours + float (max - min).Minutes / 60.)
        )
        |> fun x -> (Chart.BoxPlot(Y=x,Name=name))
    tmp
    )
|> Chart.combine
|> Chart.withAxisTitles "" "time span between first and last order (hours)"
|> Chart.show

data 
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let min = orders |> Array.map(fun x -> x.DateTime) |> Array.min
    let max = orders |> Array.map(fun x -> x.DateTime) |> Array.max 
    let employmentDuration = (max - min).Days
    let lg,col = 
        if System.DateTime.Now.AddDays(-60.) < max then 
            if data.[0].DateTime.AddDays(60.) > min then 
                "total time employed",Color.fromString "grey" 
            else
                "currently employed",Color.fromString "blue" 
        else 
            if data.[0].DateTime.AddDays(30.) > min then 
                "from start, not now",Color.fromString "green" 
            else
                "valid time span",Color.fromString "orange"
    employmentDuration,Chart.Column([name,float employmentDuration / 365.],MarkerColor=col) |> Chart.withTraceInfo(LegendGroup=lg,LegendGroupTitle=Title.init lg)
    )
|> Array.sortByDescending fst
|> Array.map snd
|> Chart.combine
|> Chart.withAxisTitles "" "employment duration (years)"
|> Chart.withTitle "employment duration"
|> Chart.withDescription [(Giraffe.ViewEngine.HtmlElements.rawText """<br>grey: total time employed<br>blue: currently employed<br>green: from start, not now<br>orange: valid time span""")]
|> Chart.show




data 
|> Array.filter (fun x -> x.DateTime.TimeOfDay < (DateTime(2023,1,1,12,0,0)).TimeOfDay)
|> Array.groupBy (fun x -> x.Name)
|> Array.map (fun (name,orders) -> 
    let tmp = 
        orders
        |> Array.groupBy (fun o -> o.DateTime.Date)
        |> Array.choose (fun (date,items) -> 
            let min = items |> Array.filter (fun t -> t.Category = Category.Coffee) 
            if min = [||] then 
                None 
            else
                let tmp = min |> Array.minBy (fun x -> x.DateTime)
                Some (System.DateTime(1900,01,01,tmp.DateTime.TimeOfDay.Hours,tmp.DateTime.TimeOfDay.Minutes,tmp.DateTime.TimeOfDay.Seconds))
        )
    name,tmp
    )
|> Array.map (fun (name,x) -> Chart.BoxPlot(Y=x,Name=name))
|> Chart.combine
|> Chart.withTitle "daily first tick distribution"
|> Chart.withAxisTitles "" "first tick"
|> Chart.show

data |> Array.filter (fun x -> x.Product.Contains "Spezi")
|> Array.map (fun x -> (string x.DateTime.Year) + (string x.DateTime.Month))
|> Array.groupBy id
|> Array.map (fun (x,items) -> x,items.Length)
|> Chart.Column
|> Chart.show



